In [ ]:
import sqlalchemy

In [ ]:
sqlalchemy.__version__
## dialect + driver :// username : password@ host:port/databes
   ## 커넥터  드라이버 sqllite  sqlite는 서버없어서 그냥 밑에처럼 간단함

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///test.db', echo = True)  ## 엔진을 통해 어떤 작업할떄 내가한 작업에 대한 에코

In [ ]:

print(engine)

In [ ]:
type(engine)

In [ ]:
## 메타데이터 >> 테이블 >> 컬럼  ## 메타데이터가 제일큼

In [ ]:
from sqlalchemy import *

In [ ]:
metadata = MetaData()

In [ ]:
users = Table('users', metadata, 
             Column('id', Integer, primary_key=True),
             Column('name', String),
             Column('fullname', String),
)


In [ ]:
addresses = Table('addresses', metadata, 
                 Column('id', Integer, primary_key=True),
                 Column('user_id', None, ForeignKey('users.id')),
                 Column('email_address', String, nullable=False)
    )

In [ ]:
metadata.tables

In [ ]:
metadata.bind

In [ ]:
metadata.create_all(engine) ## immutable alter update 이런거 안된다. 지워서 다시 엔진에 등록시켜야함    re!!
                            ## commit까지 된다. ## sql 에 join on 절에 안써도 알아서 조인이 된다.


In [ ]:
### metadata.bind

In [ ]:
insert = users.insert()
print(insert)

In [ ]:
insert = users.insert().values(name = 'kim', fullname = 'Anonymous, Kim')
print(insert)


In [ ]:
insert.compile().params   ##sql문을 자동으로 완성시켜줌 안쳐도됨, name, fullname이란 키값에 데이터 박아넣음

In [ ]:
##resultproxy --> cursor랑 똑같다.

In [ ]:
conn = engine.connect()
conn

In [ ]:
insert.bind = engine

In [ ]:
str(insert)

In [ ]:
result = conn.execute(insert)  ##Insert에서 만들어진 sql문을 실행 result 커서랑 같은거

In [ ]:
##가장 최근 작업한 id 여기서는 1이 나와야한다
result.inserted_primary_key

In [ ]:
##################### 앞에 복잡한과정 필요없이 이렇게 간단하게 insert 할수있따.
insert = users.insert()
result = conn.execute(insert, name="lee", fullname = "Unknown, Lee")
result.inserted_primary_key


In [ ]:
## 파라메터 여러개 == 쿼리 여러개 한번에 넣는거  commit 까지 다된다.
conn.execute(addresses.insert(), [
    {"user_id":1, "email_address":"anonymous.kim@"},
    {"user_id":2, "email_address":"unknown.lee@"}
])

In [ ]:
##########################################################

In [ ]:
##select

In [ ]:
query = select([users])## 다가져옴 select 쿼리


In [ ]:
users

In [ ]:
cur = conn.execute(query)

In [ ]:
for row in cur :
    print (row)

In [ ]:
cur.fetchall()

In [ ]:
##################################################################################3

In [ ]:
## + where 절한  select문  여기서 join 한거처럼 = 같다 하고 conjunction 한거
## users.c  c는 columms  users 중에 컬럼 중 아이디

In [ ]:
print(users.c.id == addresses.c.user_id)


In [ ]:
print(users.c.id == 1 )

In [ ]:
print((users.c.id == 1). compile().params)

In [ ]:
## 나머지는 pdf 보고 하기

In [ ]:
result = conn.execute(select([users]).where(users.c.id==1))
for row in result:
    print(row)
    
#result = conn.exec

In [ ]:
## pk 랑 fk를 테이블에 사전에 설정해서 join 할떄 on절은 쓸필요 없다.

In [ ]:
print(users.join(addresses))

In [ ]:
print(users.join(addresses, users.c.id == addresses.c.user_id))

In [ ]:
query = select([users.c.id, 
                users.c.fullname,
                addresses.c.email_address])\
        .select_from(users.join(addresses))
result = conn.execute(query).fetchall()
for row in result:
    print(row)

In [ ]:
## 값다시 올리고 이런거 할때metadata tables  다 완전히 지워야된다 이거 다시 

In [ ]:
#############################################################################33
## extend existing  만약에 테이블기존꺼 있으면 어떻게 할거냐

In [ ]:
import sqlalchemy

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///test2.db', echo = True)  ## 엔진을 통해 어떤 작업할떄 내가한 작업에 대한 에코

In [ ]:
from sqlalchemy import *

In [ ]:
metadata = MetaData()

In [ ]:
artist = Table("Artist", metadata,
              Column("id", Integer, primary_key=True),
                Column("name", String, nullable = False),
               extend_existing=True)

In [ ]:
album = Table("Album", metadata,
             Column("id", Integer, primary_key=True),
             Column("title", String, nullable = False),
             Column("artist_id", Integer, ForeignKey("Artist.id")),
             extend_existing=True)


In [ ]:
genre = Table("Genre", metadata,
             Column("id", Integer, primary_key = True),
             Column("name", String, nullable = False),
             extend_existing=True)

In [ ]:
track = Table("Track", metadata, 
             Column("id", Integer, primary_key=True),
             Column("title", String, nullable = False),
              Column("length", Integer, nullable = False),
              Column("rating", Integer, nullable = False),
              Column("count", Integer, nullable = False),
              Column("album_id", Integer, ForeignKey("Album.id")),
            Column("genre_id", Integer, ForeignKey("Genre.id")),
              extend_existing=True)

In [ ]:
metadata.create_all(engine)

In [ ]:
conn = engine.connect()

In [ ]:
conn.execute(artist.insert(), [
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])

In [ ]:
conn.execute(album.insert(), [
    {"title":"IV", "artist_id":1},
    {"title":"Who Made Who", "artist_id":2}
])


In [ ]:
conn.execute(genre.insert(), [
    {"name":"Rock"},
    {"name":"Metal"}
    
])

In [ ]:
conn.execute(track.insert(), [
    {"title":"Black Dog", "rating": 5, "length":297, "count":0, "album_id":1, "genre_id":1},
        {"title":"Stairway", "rating": 5, "length":482, "count":0, "album_id":1, "genre_id":1},
        {"title":"About to rock", "rating": 5, "length":313, "count":0, "album_id":2, "genre_id":2},
        {"title":"Who Made Who", "rating": 5, "length":297, "count":0, "album_id":2, "genre_id":2},
])

In [ ]:
tables = metadata.tables
for table in tables:  ## 메모리상 가져오는거
    print(table)


In [ ]:
for table in engine.table_names():   ##db에서 보여주는거   ## 둘이 일치해야함
    print(table)

In [ ]:
##############sel

In [ ]:
print(track.join(album).join(genre).join(artist))

In [ ]:
result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]).select_from(track.join(album).join(genre).join(artist)))

In [ ]:
for row in result.fetchall():
    print(row)

In [ ]:
for row in result:
    print(row)

In [ ]:
metadata.tables

In [ ]:
for table in engine.table_names():  ## 
    print(table)

In [ ]:
conn.close()
metadata.clear()         ## 메모리상에서는 사라지고 db에서만 데이터 존재하는 상태

In [ ]:
metadata = MetaData(bind = engine, reflect = True)  ## 비어있는 메타데이타에 engin에있는걸 불러오는것
metadata.reflect(bind=engine)

In [ ]:
########################### base

In [4]:
engine = create_engine('sqlite:///test5.db', echo = True)  ## 엔진을 통해 어떤 작업할떄 내가한 작업에 대한 에코

In [ ]:
engine = create_engine('sqlite:///test4.db', echo = True)  ## 엔진을 통해 어떤 작업할떄 내가한 작업에 대한 에코

In [ ]:
engine = create_engine('sqlite:///test3.db', echo = True)  ## 엔진을 통해 어떤 작업할떄 내가한 작업에 대한 에코

In [1]:
from sqlalchemy import *

In [2]:
from sqlalchemy.ext.declarative import declarative_base

In [3]:
base = declarative_base()

In [ ]:
## 애는 db에있는게 x 메모리에 있따

In [5]:
class User(base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column("passwd", String)
    
    def __repr__(self):
        return "<T'User(name = '%s', fullname = '%s', password = '%s')>" \
                % (self.name, self.fullname, self.password)

In [ ]:
User.__table__

In [ ]:
User.__repr__

In [ ]:
User.__mapper__  ## Users 라는 테이블과 매핑이 되어있다. 맵되어있다.
                 ## MVC 패턴에서 모델단, 아까는 테이블 컬럼 막추가 이런거 하는데 더 복잡

In [ ]:
base.metadata.create_all(engine)

In [ ]:
kim = User(name = 'kim', fullname = "anonymous, Kim", password = "kimbap heaven")

In [ ]:
print(kim)

In [ ]:
print(kim.id)

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.add(kim)

In [ ]:
session.add_all([
    User(name="lee", fullname = "unknow, Lee", password = '123456789a'),
    User(name="park", fullname = "nobody, Park", password = "Parking in Park")
])

In [ ]:
session.commit()

In [ ]:
#################################################################


In [ ]:
kim.password = "password"

In [ ]:
session.dirty

In [ ]:
session.is_modified(kim)

In [ ]:
kim.id

In [ ]:
print(kim.id)

In [ ]:
for row in session.query(User):
    print(type(row))
    print(row.id, row.name, row.fullname, row.password)

In [ ]:
for row in session.query(User.id, User.fullname).filter_by(name = "lee"):
    print(row)

In [6]:
from sqlalchemy.orm import *

In [7]:
class Artist(base):
    __tablename__ = "Artist"
    id = Column(Integer, primary_key = True)
    name  = Column(String)
    
    albumList = relationship("Album", back_populates="artist")
    
class Album(base):
    __tablename__ = "Album"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
             
    artist = relationship("Artist", back_populates = "albumList", uselist = False)
    trackList = relationship("Track", back_populates = "album")
        
class Genre(base):
    __tablename__ = "Genre"
    id =  Column( Integer, primary_key = True)
    name = Column( String)
                       
    trackList = relationship("Track", back_populates = "genre")     
        
class Track(base):
    __tablename__ = "Track"
    id = Column(Integer, primary_key=True)
    title =  Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column( Integer, ForeignKey("Album.id"))
    genre_id = Column(Integer, ForeignKey("Genre.id"))
       
    album = relationship("Album", back_populates = "trackList", uselist = False)
    genre = relationship("Genre", back_populates = "trackList", uselist = False)

In [ ]:
artist1 = Artist(name = '')

In [10]:
import re

In [18]:
content = "Wello World"

In [19]:
print(re.search("W", content))

<_sre.SRE_Match object; span=(0, 1), match='W'>


In [20]:
print(re.match("W", content))

<_sre.SRE_Match object; span=(0, 1), match='W'>


In [35]:
m = re.match(r"(\w+)(\w+)(\w+)", "neqqqq Newton, physicist")
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(3))
print(m.group(1,2 ))

neqqqq
neqq
q
q
('neqq', 'q')


In [ ]:
##정규식표현중요